In [1]:
from draw_diffusers import  load_model, image_process

PIPE =load_model()

c:\Users\timmy\anaconda3\envs\whiteboard\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


get model


Loading pipeline components...: 100%|██████████| 6/6 [00:01<00:00,  3.23it/s]
You have disabled the safety checker for <class 'diffusers.pipelines.controlnet.pipeline_controlnet_img2img.StableDiffusionControlNetImg2ImgPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [2]:
import pygame
from PIL import Image
from pathlib import Path
from painter import *

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
def run():
    global input
    path = Path()
    # pygame initialize
    pygame.init()

    # UI Settings

    CANVAS_W = 800
    CANVAS_H = 600

    
    W1 = 200
    H1 = 80
    BACKGROUND_COLOR = 255, 255, 225
    CANVAS_COLOR = 255, 255, 255
    CAPTION = "BCI Drawing"
    MARGIN1 = 10
    MARGIN2 = 10
    BRUSH_COLOR = BLACK

    WIDTH = CANVAS_W * 2 + W1 + MARGIN1 * 4
    HEIGHT = CANVAS_H + MARGIN1 * 2


    pygame.display.set_caption(CAPTION)
    screen = pygame.display.set_mode((WIDTH, HEIGHT))

    GENERATED_IMAGE = None

    # build objects
    canvas = Canvas(
        color=CANVAS_COLOR,
        rect=(
            MARGIN1 * 2 + W1,  # x coordinate
            MARGIN1,  # y coordinate
            CANVAS_W,  # width
            CANVAS_H  # height
        )
    )

    canvas_2 = Canvas(
        color=CANVAS_COLOR,
        rect=(
            MARGIN1 * 3 + W1 + CANVAS_W,  # x coordinate
            MARGIN1,  # y coordinate
            CANVAS_W,  # width
            CANVAS_H  # height
        )
    )

    current_time = pygame.time.get_ticks()
    button_back = Button(
        name="Back Button",
        color=RED,
        rect=(
            MARGIN1,  # x coordinate
            MARGIN1,  # y coordinate
            (W1 - MARGIN2) / 2,  # width
            H1  # height
        ),
        time=current_time,
        target_f=6,  # target frequency
        img=pygame.image.load(path / 'icons' / 'back.png'),
    )
    button_next = Button(
        name="Next Button",
        color=GREEN,
        rect=(
            MARGIN1 + MARGIN2 + (W1 - MARGIN2) / 2,  # x coordinate
            MARGIN1,  # y coordinate
            (W1 - MARGIN2) / 2,  # width
            H1  # height
        ),
        time=current_time,
        target_f=7,  # target frequency
        img=pygame.image.load(path / 'icons' / 'next.png'),
    )
    button_comfirm = Button(
        name="Confirm Button",
        color=YELLOW,
        rect=(
            MARGIN1,  # x coordinate
            HEIGHT - MARGIN1 - H1,  # y coordinate
            W1,  # width
            H1  # height
        ),
        time=current_time,
        target_f=10,  # target frequency
        text="Confirm",
    )
    button_list = [button_back, button_next, button_comfirm]

    # --- Clock, fps ---
    """
    fps = 60
    clock = pygame.time.Clock()
    """
    # --- events ---
    running = True
    is_flashing = True

    while running:
        # clock.tick(fps)

        # --- events ---
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.VIDEORESIZE:
                print("The Pygame window is now " + str(event.w) + " pixels wide and " + str(event.h) + " pixels high")
                WIDTH, HEIGHT = event.size
                # if WIDTH < min_width:
                #     WIDTH = min_width
                # if HEIGHT < min_height:
                #     HEIGHT = min_height
                screen = pygame.display.set_mode((WIDTH, HEIGHT), HWSURFACE | DOUBLEBUF | RESIZABLE)
                CANVAS_SIZE = (WIDTH - MARGIN1 * 2) / 2, HEIGHT - MARGIN1 * 2
                screen.fill(BACKGROUND_COLOR)
                canvas = pygame.transform.scale(canvas, CANVAS_SIZE)
                canvas_2 = pygame.transform.scale(canvas_2, CANVAS_SIZE)
            elif event.type == pygame.KEYDOWN:
                # Save canvas when "S" is pressed
                if event.key == pygame.K_s:
                    canvas.save()
                # Reset canvas when "R" is pressed
                if event.key == pygame.K_r:
                    canvas.reset()
                # Open/Close button blinking when "F" is pressed
                if event.key == pygame.K_f:
                    is_flashing = not is_flashing
            if event.type == MOUSEBUTTONDOWN:
                print('mouse down')
                mouse_pos = pygame.mouse.get_pos()
                for button in button_list:
                    if button.get_rect().collidepoint(mouse_pos):
                        print(f"--- {button.name} got pressed ---    \r", end='\b')
            
            if event.type == MOUSEBUTTONUP:
                print('mouse up')
                # GENERATED_IMAGE = pygame.image.load("../draw/img/input_5.png").convert()
                # GENERATED_IMAGE = pygame.transform.scale(GENERATED_IMAGE, (CANVAS_W,CANVAS_H))
                
                string_img = pygame.image.tostring(canvas.export_img(),'RGB')
                pil_img = Image.frombytes("RGB",(CANVAS_W,CANVAS_H),string_img)
                output = image_process(pipe=PIPE,image=pil_img)
                output_bytes = output.tobytes()
                mode = output.mode
                size = output.size
                GENERATED_IMAGE = pygame.image.fromstring(output_bytes, size, mode)

        if pygame.mouse.get_pressed()[0]:
            mouse_pos = pygame.mouse.get_pos()
            if canvas.get_rect().collidepoint(mouse_pos):
                canvas.add_brush(
                    color=BRUSH_COLOR,
                    position=mouse_pos,
                    size=10
                )

               
        # --- updates ---
        if is_flashing:
            current_time = pygame.time.get_ticks()
            for button in button_list:
                button.update(current_time)
        else:
            for button in button_list:
                button.show = True

        # --- draws ---
        screen.fill(BACKGROUND_COLOR)
        for button in button_list:
            button.draw(screen)
        canvas.draw(screen)
        
        if (GENERATED_IMAGE != None):
            screen.blit(GENERATED_IMAGE, (MARGIN1 * 3 + W1 + CANVAS_W, MARGIN1)) 
        else:
            canvas_2.draw(screen)
        
        # button_back.observe()     # Uncomment to observe actual frequency of button
        pygame.display.flip()

    print("Ended" + " " * 100)
    pygame.quit()

In [4]:
run()

mouse down
mouse up
image process


100%|██████████| 16/16 [00:02<00:00,  5.97it/s]


Ended                                                                                                    
